# Make movies from LLC

In [1]:
# Load some useful modules 
import numpy as np
import xarray as xr
import xrft
from xmitgcm import llcreader
from matplotlib import pyplot as plt
from xmovie import Movie
import cmocean.cm as cm

In [2]:
%matplotlib inline
# this needs to be commented out so that xmovie does not crash

In [3]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml")

In [4]:
# Get variables from catalog
sst = cat.LLC4320_SST.to_dask()
sss = cat.LLC4320_SSS.to_dask()
ssh = cat.LLC4320_SSH.to_dask()
u = cat.LLC4320_SSU.to_dask()
v = cat.LLC4320_SSV.to_dask()

In [5]:
coords = cat.LLC4320_grid.to_dask()

In [6]:
u.U.attrs = {'long_name': 'Zonal Velocity (m/s)', 
            'mate': 'V', 'units':'m/s'}
v.V.attrs = {'long_name': 'Meridional Velocity (m/s)', 
            'mate': 'U', 'units':'m/s'}

In [7]:
ds = xr.merge([ u, v, sst, ssh, sss.SSS, coords])

In [8]:
ds_ll = llcreader.llcmodel.faces_dataset_to_latlon(ds, 
                metric_vector_pairs=[])

In [9]:
import xgcm
grid = xgcm.Grid(ds_ll.drop(['k', 'k_p1']), periodic='X')

In [10]:
from fastjmd95 import rho
ds_ll['SSD'] = xr.apply_ufunc(rho, 
                        ds_ll.SSS, ds_ll.SST, 0, 
                        dask='parallelized',
                        output_dtypes=[float,]).rename('SSD')

#import gsw
#ds['Spice'] = xr.apply_ufunc(gsw.spiciness0, 
#                        ds.SSS, ds.SST,  
#                        dask='parallelized', output_dtypes=[float,]).rename('Spice')

In [11]:
sel_XC = np.logical_and(ds_ll.XC>50, ds_ll.XC<100 ) 
sel_XG = np.logical_and(ds_ll.XG>50, ds_ll.XG<100 )
sel_YC = np.logical_and(ds_ll.YC>0, ds_ll.YC<30 )
sel_YG = np.logical_and(ds_ll.YG>0, ds_ll.YG<30 )

In [12]:
XC = ds_ll.XC.where(sel_XC & sel_YC, drop=True)
YC = ds_ll.YC.where(sel_XC & sel_YC, drop=True)
XCmean = XC.mean(['j'])
YCmean = YC.mean(['i'])

Xmax = XC.max(['j'])
Xmin = XC.min(['j'])
Ymax = YC.max(['i'])
Ymin = YC.min(['i'])

In [13]:
XG = coords.XG.where(sel_XG & sel_YG, drop=True)
YG = coords.YG.where(sel_XG & sel_YG, drop=True)
XGmean = XG.mean(['j_g'])
YGmean = YG.mean(['i_g'])

## Surface TimeSeries 

## Movie of Surface Tracers

In [14]:
SSH_IO = ds_ll.Eta.where(sel_XC & sel_YC, drop=True)
SST_IO = ds_ll.SST.where(sel_XC & sel_YC, drop=True)
SSS_IO = ds_ll.SSS.where(sel_XC & sel_YC, drop=True)
SSD_IO = ds_ll.SSD.where(sel_XC & sel_YC, drop=True)

/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/core/indexing.py:1367: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/core/indexing.py:1367: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(sl

In [15]:
ds_IO = xr.merge([SST_IO, SSS_IO, SSD_IO, SSH_IO])

In [45]:
ds_IO.iter.values

array([  10368,   10512,   10656, ..., 1310256, 1310400, 1310544])

In [17]:
ds_IO = ds_IO.chunk({"time":1,"i":-1, "j":-1})

In [55]:
def save_image(block):
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    #if sum(block.shape) > 0:
    # workaround 1:
    # xarray passes a zero shaped array to infer what this function returns. 
    # we can't run plot, so avoid doing that
    f = plt.figure()
    ax = f.subplots(1, 1)

    # xarray plotting goodness is available here!
    block.SST.plot(ax=ax, vmin=5, vmax=28, 
                   cmap=cm.thermal, cbar_kwargs={"extend": "both"})

    # on pangeo.io, this will need some tweaking to work with gcsfs.
    # haven't tried that. On cheyenne, it works beautifully.
    f.savefig(f"temp/{block.iter.values}.png", dpi=180)
    plt.close(f)

    # workaround 2:
    # map_blocks expects to receive an xarray thing back.
    # Just send back one value. If we send back "block" that's like computing the whole dataset!
    return block.time

In [49]:
save_image(ds_IO.isel(time=0))

<xarray.DataArray 'time' ()>
array('2011-09-13T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2011-09-13
    PHrefC   float32 15.4017
    Z        float32 -1.57
    Zl       float32 -1.0
    Zu       float32 -2.14
    drF      float32 1.14
    iter     int64 dask.array<chunksize=(), meta=np.ndarray>
    k        int64 1
    k_l      int64 1
    k_u      int64 1
Attributes:
    axis:           T
    long_name:      Time
    standard_name:  time

In [70]:
temp_array = ds_IO.isel(time=0).XC[0,0]

In [71]:
temp_array

<xarray.DataArray 'XC' ()>
dask.array<getitem, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    j        int64 7920
    time     datetime64[ns] 2011-09-13
    i        int64 4224
    CS       float32 dask.array<chunksize=(), meta=np.ndarray>
    Depth    float32 dask.array<chunksize=(), meta=np.ndarray>
    PHrefC   float32 15.4017
    SN       float32 dask.array<chunksize=(), meta=np.ndarray>
    XC       float32 dask.array<chunksize=(), meta=np.ndarray>
    YC       float32 dask.array<chunksize=(), meta=np.ndarray>
    Z        float32 -1.57
    Zl       float32 -1.0
    Zu       float32 -2.14
    drF      float32 1.14
    hFacC    float32 dask.array<chunksize=(), meta=np.ndarray>
    iter     int64 dask.array<chunksize=(), meta=np.ndarray>
    k        int64 1
    k_l      int64 1
    k_u      int64 1
    rA       float32 dask.array<chunksize=(), meta=np.ndarray>
Attributes:
    coordinate:     YC XC
    long_name:      longitude
    standard_name:  longitude
    units:          degrees_east

In [69]:
ds_IO.isel(time=slice(0,4)).map_blocks(save_image, template=temp_array)

<xarray.DataArray 'XC' (j: 1639, i: 2400)>
dask.array<save_image-db734807dbe70d92e4dfffb602aae6cd-<this, shape=(1639, 2400), dtype=float32, chunksize=(1639, 2400), chunktype=numpy.ndarray>
Coordinates:
  * j        (j) int64 7920 7921 7922 7923 7924 ... 9554 9555 9556 9557 9558
  * i        (i) int64 4224 4225 4226 4227 4228 ... 6619 6620 6621 6622 6623
    time     datetime64[ns] dask.array<chunksize=(), meta=np.ndarray>
    CS       (j, i) float32 dask.array<chunksize=(1639, 2400), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(1639, 2400), meta=np.ndarray>
    PHrefC   float32 dask.array<chunksize=(), meta=np.ndarray>
    SN       (j, i) float32 dask.array<chunksize=(1639, 2400), meta=np.ndarray>
    XC       (j, i) float32 dask.array<chunksize=(1639, 2400), meta=np.ndarray>
    YC       (j, i) float32 dask.array<chunksize=(1639, 2400), meta=np.ndarray>
    Z        float32 dask.array<chunksize=(), meta=np.ndarray>
    Zl       float32 dask.array<chunksize=(), meta=np.ndarray>
    Zu       float32 dask.array<chunksize=(), meta=np.ndarray>
    drF      float32 dask.array<chunksize=(), meta=np.ndarray>
    hFacC    (j, i) float32 dask.array<chunksize=(1639, 2400), meta=np.ndarray>
    iter     int64 dask.array<chunksize=(), meta=np.ndarray>
    k        int64 dask.array<chunksize=(), meta=np.ndarray>
    k_l      int64 dask.array<chunksize=(), meta=np.ndarray>
    k_u      int64 dask.array<chunksize=(), meta=np.ndarray>
    rA       (j, i) float32 dask.array<chunksize=(1639, 2400), meta=np.ndarray>
Attributes:
    coordinate:     YC XC
    long_name:      longitude
    standard_name:  longitude
    units:          degrees_east

In [19]:
def custom_plotfunc(ds_sogos, fig, tt):
    
    ds_sogos = ds_sogos.isel(i=slice(0,-1,4),j=slice(0,-1,4))
    XC = ds_sogos.XC
    YC = ds_sogos.YC 
    
    ax = fig.subplots(2, 2)

    ax1= ax[0,0]
    ax2= ax[0,1]
    ax3= ax[1,0]
    ax4= ax[1,1]

    p1 = ax1.pcolormesh(XC, YC, ds_sogos.SST.isel(time=tt), cmap=cm.thermal,
                        vmin=24.5, vmax=30.5)
    
    cbar1 = fig.colorbar(p1 , ax=ax1)
    cbar1.ax.set_ylabel('Temp')
    ax1.set_aspect(1.25)
    ax1.set_title(str((ds_sogos.time.isel(time=tt)).dt.strftime("%-H, %b %d, %y").values))

    p2 = ax2.pcolormesh(XC, YC, ds_sogos.SSS.isel(time=tt), cmap=cm.haline,
                       vmin=29, vmax=36.8)
        
    cbar2 = fig.colorbar(p2 , ax=ax2)
    cbar2.ax.set_ylabel('Salt')
    ax2.set_aspect(1.25)
    ax2.set_title(str((ds_sogos.time.isel(time=tt)).dt.season.values))

    p3 = ax3.pcolormesh(XC, YC, ds_sogos.SSD.isel(time=tt), cmap=cm.dense,
                       vmin=1017, vmax=1025)
    
    cbar3 = fig.colorbar(p3 , ax=ax3)
    cbar3.ax.set_ylabel('Density')
    ax3.set_aspect(1.25)
    #ax3.set_title(str((ds_sogos.time.isel(time=tt)).dt.strftime("%b %d, %y").values))

    p4 = ax4.pcolormesh(XC, YC, ds_sogos.Eta.isel(time=tt), cmap=cm.balance,
                       vmin=-2, vmax=2)
        
    cbar4 = fig.colorbar(p4 , ax=ax4)
    cbar4.ax.set_ylabel('SSH')
    ax4.set_aspect(1.25)
#    ax4.set_title(str((ds_sogos.time.isel(time=tt)).dt.season.values))
    
    fig.tight_layout()


In [20]:
fig = plt.figure(figsize=(12,10))
custom_plotfunc(ds_IO, fig, 6000)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:23: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:31: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is depre

In [21]:
mov_custom = Movie(ds_IO.isel(time=slice(0,-1,2)), 
                   custom_plotfunc, 
                   input_check=False)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:23: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:31: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is depre

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
mov_custom.save('movie_IO/movie_IO.mp4'
                ,progress=True,remove_frames=False, overwrite_existing=True,
               remove_movie=False, start_frame=0)

movie_IO/movie_IO.mp4


_call non-retriable exception: 
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/gcsfs/core.py", line 507, in _call
    self.validate_response(status, contents, json, path, headers)
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/gcsfs/core.py", line 1230, in validate_response
    raise HttpError({"code": status})
gcsfs.utils.HttpError


HttpError: 

## Movie of surface kinematics

In [12]:
KE = 0.5*(grid.interp(ds.U,'X',boundary='extend')**2 + grid.interp(ds.V,'Y',boundary='extend')**2) 

zeta = (-grid.diff(ds.U * coords.dxC, 'Y', boundary='extend') + grid.diff(ds.V * coords.dyC, 'X', boundary='extend'))/coords.rAz
zeta = grid.interp(grid.interp(zeta, 'X', boundary='extend'), 'Y', boundary='extend')

strain1 = (grid.diff(ds.U * coords.dyG, 'X', boundary='extend') - grid.diff(ds.V * coords.dxG, 'Y',boundary='extend')) / coords.rA
strain2 = (grid.diff(ds.U * coords.dxC, 'Y', boundary='extend') + grid.diff(ds.V * coords.dyC, 'X', boundary='extend'))/coords.rAz
strain2 = grid.interp(grid.interp(strain2, 'X', boundary='extend'), 'Y', boundary='extend')
strain = (strain1**2 + strain2**2)**0.5

gradD = (grid.interp(grid.diff(ds.SSD,'X',boundary='extend')/coords.dxC, 'X', boundary='extend')**2 +
         grid.interp(grid.diff(ds.SSD,'Y',boundary='extend')/coords.dyC, 'Y', boundary='extend')**2)**0.5

In [13]:
zeta_sogos = zeta.where(sel_XC & sel_YC, drop=True)
strain_sogos = strain.where(sel_XC & sel_YC, drop=True)
gradD_sogos = gradD.where(sel_XC & sel_YC, drop=True)
KE_sogos = KE.where(sel_XC & sel_YC, drop=True)
SSD_sogos = ds.SSD.where(sel_XC & sel_YC, drop=True)

In [14]:
f = 2*(2*np.pi/24/3600)*np.sin(-55*np.pi/360)

ds_kinem_sogos = xr.merge([zeta_sogos.rename('Vorticity')/f,
                           strain_sogos.rename('Strain')/(-f), 
                           gradD_sogos.rename('rho_grad'),
                           KE_sogos.rename('KE'),
                          SSD_sogos])
# f=2*omega*sin(theta)


In [15]:
def custom_plotfunc2(ds_kinem_sogos, fig, tt):
    
    ds_kinem_sogos = ds_kinem_sogos.isel(i=slice(0,-1,1),j=slice(0,-1,1))

    XC = ds_kinem_sogos.XC
    YC = ds_kinem_sogos.YC 

    ax = fig.subplots(2, 2)

    ax1= ax[0,0]
    ax2= ax[0,1]
    ax3= ax[1,0]
    ax4= ax[1,1]

    p1 = ax1.pcolormesh(XC, YC, ds_kinem_sogos.KE.isel(time=tt), cmap=cm.speed_r, vmin=0, vmax=0.8)
    ax1.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax1.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar1 = fig.colorbar(p1 , ax=ax1)
    cbar1.ax.set_ylabel('KE (m/s)')
    ax1.set_aspect(1.25)
    ax1.set_title(str((ds_kinem_sogos.time.isel(time=tt)).dt.strftime("%b %d, %y").values))

    p2 = ax2.pcolormesh(XC, YC, ds_kinem_sogos.Vorticity.isel(time=tt), cmap=cm.curl, vmin=-1, vmax=1)
    ax2.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax2.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar2 = fig.colorbar(p2 , ax=ax2)
    cbar2.ax.set_ylabel('Vorticity/f')
    ax2.set_aspect(1.25)
    ax2.set_title(str((ds_kinem_sogos.time.isel(time=tt)).dt.season.values))

    p3 = ax3.pcolormesh(XC, YC, ds_kinem_sogos.Strain.isel(time=tt), cmap=cm.amp, vmin=0, vmax=1)
    ax3.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax3.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar3 = fig.colorbar(p3, ax=ax3)
    cbar3.ax.set_ylabel('Strain/f')
    ax3.set_aspect(1.25)

    p4 = ax4.pcolormesh(XC, YC, ds_kinem_sogos.rho_grad.isel(time=tt), cmap=cm.ice, vmin=0.1e-5, vmax= 3.5e-5)
    ax4.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax4.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar4 = fig.colorbar(p4, ax=ax4)
    cbar4.ax.set_ylabel(r'$|\nabla \rho|$')
    ax4.set_aspect(1.25)

    fig.tight_layout()

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
mov_custom_2 = Movie(ds_kinem_sogos.isel(time=slice(0,-1,24)), 
                   custom_plotfunc2, 
                   input_check=False)

In [18]:
mov_custom_2.save('movie_KZSG/movie_KZSG.mp4'
                ,progress=True,remove_frames=False,
                remove_movie=False, start_frame=113)

movie_KZSG/movie_KZSG.mp4



Movie created at movie_KZSG.mp4


In [20]:
import os
dirname = os.path.dirname('movie_KZSG/movie_KZSG.mp4')

In [21]:
dirname

'movie_KZSG'

In [25]:
#from xmovie.core import frame_save

def frame_save(fig, frame, odir=None, frame_pattern="frame_%05d.png", dpi=100):
    fig.savefig(
        os.path.join(odir, frame_pattern % (frame)),
        dpi=dpi,
        facecolor=fig.get_facecolor(),
        transparent=True,
    )
    # I am trying everything to *wipe* this figure, hoping that it could
    # help with the dask glitches I experienced earlier.
    # TBD if this is all needed...how this might affect performance.
    plt.close('all')
    #del fig
    #gc.collect(2)


In [26]:
%matplotlib notebook

In [27]:
for fi in range(67,75):
    fig, ax, pp = mov_custom_2.render_frame(fi)
    frame_save(fig, fi, odir=dirname, frame_pattern=mov_custom_2.frame_pattern, 
               dpi=mov_custom_2.dpi )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>